In [1]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()
model.load_model("catboost_test.cbm")
#directly using the line below may not fit the mql5 format
#model.save_model('catboost_model.onnx', format='onnx')

In [2]:
from onnx.helper import get_attribute_value
import onnxruntime as rt
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
)  # noqa
from skl2onnx.common.data_types import (
    FloatTensorType,
    Int64TensorType,
    guess_tensor_type,
)
from skl2onnx._parse import _apply_zipmap, _get_sklearn_operator_name
from catboost import CatBoostClassifier
from catboost.utils import convert_to_onnx_object

def skl2onnx_parser_castboost_classifier(scope, model, inputs, custom_parsers=None):
    
    options = scope.get_options(model, dict(zipmap=True))
    no_zipmap = isinstance(options["zipmap"], bool) and not options["zipmap"]

    alias = _get_sklearn_operator_name(type(model))
    this_operator = scope.declare_local_operator(alias, model)
    this_operator.inputs = inputs

    label_variable = scope.declare_local_variable("label", Int64TensorType())
    prob_dtype = guess_tensor_type(inputs[0].type)
    probability_tensor_variable = scope.declare_local_variable(
        "probabilities", prob_dtype
    )
    this_operator.outputs.append(label_variable)
    this_operator.outputs.append(probability_tensor_variable)
    probability_tensor = this_operator.outputs

    if no_zipmap:
        return probability_tensor

    return _apply_zipmap(
        options["zipmap"], scope, model, inputs[0].type, probability_tensor
    )


def skl2onnx_convert_catboost(scope, operator, container):
    """
    CatBoost returns an ONNX graph with a single node.
    This function adds it to the main graph.
    """
    onx = convert_to_onnx_object(operator.raw_operator)
    opsets = {d.domain: d.version for d in onx.opset_import}
    if "" in opsets and opsets[""] >= container.target_opset:
        raise RuntimeError("CatBoost uses an opset more recent than the target one.")
    if len(onx.graph.initializer) > 0 or len(onx.graph.sparse_initializer) > 0:
        raise NotImplementedError(
            "CatBoost returns a model initializers. This option is not implemented yet."
        )
    if (
        len(onx.graph.node) not in (1, 2)
        or not onx.graph.node[0].op_type.startswith("TreeEnsemble")
        or (len(onx.graph.node) == 2 and onx.graph.node[1].op_type != "ZipMap")
    ):
        types = ", ".join(map(lambda n: n.op_type, onx.graph.node))
        raise NotImplementedError(
            f"CatBoost returns {len(onx.graph.node)} != 1 (types={types}). "
            f"This option is not implemented yet."
        )
    node = onx.graph.node[0]
    atts = {}
    for att in node.attribute:
        atts[att.name] = get_attribute_value(att)
    container.add_node(
        node.op_type,
        [operator.inputs[0].full_name],
        [operator.outputs[0].full_name, operator.outputs[1].full_name],
        op_domain=node.domain,
        op_version=opsets.get(node.domain, None),
        **atts,
    )


update_registered_converter(
    CatBoostClassifier,
    "CatBoostCatBoostClassifier",
    calculate_linear_classifier_output_shapes,
    skl2onnx_convert_catboost,
    parser=skl2onnx_parser_castboost_classifier,
    options={"nocl": [True, False], "zipmap": [True, False, "columns"]},
)

In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
data = pd.read_csv("merged_data.csv",index_col=0)
XX = data.drop(columns=['bin'])
yy = data['bin']
y = yy.values
X = XX.values
pd.DataFrame(X,y)

0      1      2     3     4     5       6        7     8         9   \
1   1.04  21.90  50.21  1.17 -0.79  1.31   23.92    37.01  0.40     -1.11   
0   0.47  15.41  23.42  1.09  0.45  1.35  213.35   149.19  0.54     15.30   
1   0.16  21.22  19.09  0.79 -0.01  0.29   17.60   -26.49  0.58      0.00   
0   0.36  24.87  36.80  1.12 -1.85  2.35  111.08   -12.15  0.59     27.45   
0  -1.06  26.04  37.18  1.62 -3.73 -2.03 -179.22  -194.41  0.28    -82.40   
..   ...    ...    ...   ...   ...   ...     ...      ...   ...       ...   
0   2.87  55.25  24.79  4.81  1.48  8.61   88.53  1850.90  0.93  12756.62   
0  -3.46  43.96  24.31  4.96 -7.14 -3.14 -146.26 -4657.47  0.15  -2043.66   
0  -1.78  14.58  13.07  6.75 -4.45 -1.06  -61.29 -3923.85  0.36   -844.57   
1   2.28  27.60  15.79  6.21  0.99  4.41   64.62  5012.92  0.72   1147.26   
1  -1.87  35.75  28.03  5.41 -8.73  1.35 -138.09  -512.47  0.24  -7245.10   

    ...    15     16    17    18     19     20       21      22    23    24  
1   ... -0.35  43.21  0.11  1.11  58.25 -54.00   436.34    24.0   2.0  0.46  
0   ... -0.16  55.63  0.07  0.58  81.58 -13.33   428.13   153.0  18.0  0.07  
1   ... -0.16  50.06 -0.24  0.42  37.14 -62.50   427.95    13.0   6.0  0.05  
0   ... -0.45  50.43  0.02  0.61  47.37 -42.86   422.60   223.0  15.0  0.26  
0   ... -1.25  30.25 -0.24  1.77   7.59 -93.75   421.83   206.0  19.0  0.29  
..  ...   ...    ...   ...   ...    ...    ...      ...     ...   ...   ...  
0   ...  1.23  61.03  0.33  8.47  78.99 -10.48  2732.35  9208.0  16.0  0.02  
0   ... -0.97  37.75 -0.15  3.68  19.40 -75.74  2731.67  4119.0   8.0  0.00  
0   ... -0.04  47.88 -0.05  4.89  19.49 -62.49  2732.36  4753.0  14.0  0.00  
1   ...  2.21  56.56  0.06  4.77  71.53 -22.33  2734.09  4120.0  21.0  0.05  
1   ... -0.15  39.33 -0.14  3.96  43.69 -91.21  2776.13  8057.0  15.0  0.21  

[4638 rows x 25 columns]

In [4]:
model_onnx = convert_sklearn(
    model,
    "catboost",
    [("input", FloatTensorType([None, X.shape[1]]))],
    target_opset={"": 12, "ai.onnx.ml": 2},
)

# And save.
with open("CatBoost_test.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())